### Zadanie

Cel: Ekstrakcja twarzy osoby starszej (70+) z jednego zdjęcia i wklejenie twarzy gdzie indziej.

TODO:
- [x] Stworzyć repozytorium zawierające notebooka
- [x] Przeszukać internet w poszukiwaniu odpowiedniego modelu
- [x] Skompletowanie datasetu (15-30 zdjęć) - również przypadki skrajne, ale 1 osoba na zdjęciu max 15 stopni odchylenia od frontu
- [x] Anotacja danych
- [ ] Przekształcenie w maski
- [ ] Analiza działania na zdjęciach zawierających osoby starsze
- [ ] Zaproponowanie rozwiązania problemów powstałych na prezentowanym datasecie

### Analiza problemu


Modele:
- [Caffe model, only face](https://github.com/YuvalNirkin/face_segmentation) - caffe model
- [PyTorch, many elements](https://github.com/zllrunning/face-parsing.PyTorch) - no requirements specified (but latest version works! yay!)
- [Keras, many elements, many ported models](https://github.com/shaoanlu/face_toolbox_keras) - with specified requirements not working
- [PyTorch, many elements, part of GAN](https://github.com/switchablenorms/CelebAMask-HQ/tree/master/face_parsing) - pytorch 0.4.1 <3


Datasety:
- [CelebAMask-HQ](https://github.com/switchablenorms/CelebAMask-HQ)
- [Helen](https://github.com/JPlin/Relabeled-HELEN-Dataset?utm_source=catalyzex.com)


Wybrano [PyTorch, many elements](https://github.com/zllrunning/face-parsing.PyTorch) ze względu na uniwersalność (co powinno być wykrywane ostatecznie jest kwestią biznesową)

In [11]:
# Image pre-processing - resize all images to 512x512

import cv2
import os

path_to_dataset = '../my_dataset/'
output_dataset = '../output_dataset/'

for image_path in os.listdir(path_to_dataset):
    print(image_path)
    img = cv2.imread(os.path.join(path_to_dataset, image_path))
    img = cv2.resize(img, (512, 512))
    cv2.imwrite(os.path.join(output_dataset, image_path), img)

old_man_dark.jpg
very_old_man_02.jpeg
old_man.jpg
very_old_man_02.jpg
very_old_woman_no_color.jpg
old_woman_dark.jpg
old_man_in_mask.jpg
old_woman_with_mask.jpg
old_woman.jpg
old_asian_man.jpg
very_old_woman.jpg
strange_face_expression.jpg
old_black_man.jpg
very_old_woman_02.jpg
woman_glasses.jpg
old_man_mask.jpg
very_very_old_woman.jpg
very_old_man.jpg
indian_man.jpg
man_not_frontal.jpeg
old_man_02.jpg
old_man_03.jpg


Do labelowania danych posłuży narzędzie Labelbox. 

Założenie: Interesuje nas wykrywanie twarzy w całości. W związku z tym z wyłączeniem:
- włosów
- szyji
- uszów
- maseczki
- brody (za wyjątkiem tej, która zakrywa rysy twarzy)

Za to z włączeniem:
- Maski pokrywającej całą twarz
- Okularami


Dataset dołączono do gita, w celu możliwości zapoznania się zposzczególnymi krokami
